# NER to LLM

In [37]:
import os
from lxml import etree
import pandas as pd
from openai import OpenAI
import requests
from ollama import chat
import json
import uuid
from tqdm import tqdm
from collections import Counter
from dotenv import load_dotenv

## Extracting XML elements

In [ ]:
# to install ollama python library before
!OLLAMA_HOST=127.0.0.1:11435 ollama serve

In [2]:
def extract_elements(xml_file):
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
    with open(xml_file, 'rb') as file:
        tree = etree.parse(file)
    
    elements = {
        'placeName': tree.xpath('/tei:TEI/tei:text/tei:body/tei:sp/tei:ab/tei:seg/tei:reg/tei:placeName', namespaces=namespaces),
        'persName': tree.xpath('/tei:TEI/tei:text/tei:body/tei:sp/tei:ab/tei:seg/tei:reg/tei:persName', namespaces=namespaces),
        'target': tree.xpath('//tei:ptr/@target', namespaces=namespaces),
        'author': tree.xpath('//tei:author/tei:persName/text()', namespaces=namespaces),
        'title': tree.xpath('//tei:title/text()', namespaces=namespaces),
        'pubPlace': tree.xpath('//tei:pubPlace/text()', namespaces=namespaces),
        'date': tree.xpath('//tei:TEI/tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl/tei:date/@when', namespaces=namespaces)
    }
    
    return tree, elements

In [3]:
def process_xml_folder(folder_path):
    data_place = []
    data_pers = []
    
    for filename in tqdm(os.listdir(folder_path), desc="Processing XML files"):
        if filename.endswith(".xml"):
            file_path = os.path.join(folder_path, filename)
            tree, elements = extract_elements(file_path)
            
            placeNames = [el.text for el in elements['placeName']]
            persNames = [el.text for el in elements['persName']]
            target = elements['target'][0] if elements['target'] else None
            author = elements['author'][0] if elements['author'] else None
            title = elements['title'][0] if elements['title'] else None
            pubPlace = elements['pubPlace'][0] if elements['pubPlace'] else None
            date = elements['date'][0] if elements['date'] else None
            
            placeName_counts = Counter(placeNames)
            persName_counts = Counter(persNames)
            
            for placeName in set(placeNames):
                data_place.append({
                    'target': target,
                    'title': title,
                    'pubPlace': pubPlace,
                    'date': date,
                    'author': author,
                    'place': placeName,
                    'placeName_num': placeName_counts[placeName]
                })
                
            for persName in set(persNames):
                data_pers.append({
                    'target': target,
                    'title': title,
                    'pubPlace': pubPlace,
                    'date': date,
                    'author': author,
                    'person': persName,
                    'persName_num': persName_counts[persName]
                })
    
    return data_place, data_pers

In [4]:
def create_dataframes(data_place, data_pers):
    df_place = pd.DataFrame(data_place)
    df_pers = pd.DataFrame(data_pers)
    
    df_place = df_place.drop_duplicates().sort_values(by='placeName_num', ascending=False)
    df_pers = df_pers.drop_duplicates().sort_values(by='persName_num', ascending=False)
    
    return df_place, df_pers

In [5]:
folder_path = '/Users/nicola/Documents/Academia/Projects/TextEnt/Processing/NER'

data_place, data_pers = process_xml_folder(folder_path)
df_place, df_pers = create_dataframes(data_place, data_pers)

Processing XML files: 100%|██████████| 594/594 [00:02<00:00, 267.59it/s]


In [6]:
df_place["place"] = df_place["place"].str.lower()
df_place.head(10)

,target,title,pubPlace,date,author,place,placeName_num
805,http://catalogue.bnf.fr/ark:/12148/cb447294587,"Le Théâtre d'Alexandre Hardy, Parisien, tome q...",Grenoble,1626,"Hardy, Alexandre",clitie,108
5273,http://catalogue.bnf.fr/ark:/12148/cb38652730w,Athénaïs...,Grenoble,1700,"Pradon, Nicolas",rome,78
2942,http://catalogue.bnf.fr/ark:/12148/cb35285434p,Le Dictateur romain : tragédie dédiée à Mgr le...,Grenoble,1647,"Mareschal, André",rome,77
2798,http://catalogue.bnf.fr/ark:/12148/cb30881255g,"Le dictateur romain , tragédie...",Grenoble,1646,"Mareschal, André",rome,73
3815,http://catalogue.bnf.fr/ark:/12148/cb44729688q,"Le Jeune Marius, tragédie",Grenoble,1669,"Boyer, Claude",rome,69
5252,http://catalogue.bnf.fr/ark:/12148/cb38652730w,Athénaïs...,Grenoble,1700,"Pradon, Nicolas",carthage,67
876,http://catalogue.bnf.fr/ark:/12148/cb38650322p,"Horace, tragédie par le Sieur Corneille...",Grenoble,1647,"Corneille, Pierre",rome,67
4462,http://catalogue.bnf.fr/ark:/12148/cb303328602,"Oeuvres poétiques du sieur Desmarets,...",Grenoble,1641,"Desmarets de Saint-Sorlin, Jean",europe,67
3062,http://catalogue.bnf.fr/ark:/12148/cb352854320,Europe : comédie héroïque,Grenoble,1645,"Desmarets de Saint-Sorlin, Jean",europe,64
4556,http://catalogue.bnf.fr/ark:/12148/cb38650916d,"La Critique de l'Escole des femmes, comédie pa...",Grenoble,1663,Molière,uranie,63


In [7]:
df_pers["person"] = df_pers["person"].str.lower()
df_pers.head()

,target,title,pubPlace,date,author,person,persName_num
13386,http://catalogue.bnf.fr/ark:/12148/cb309580795,Les oeuvres de Monsieur Molière...,Grenoble,1666,Molière,mascarille,412
35278,http://catalogue.bnf.fr/ark:/12148/cb33624900p,Le Théâtre italien ou le recueil de toutes les...,Grenoble,1694,None,colombine,398
22159,http://catalogue.bnf.fr/ark:/12148/cb38650754d,Les Oeuvres de Monsieur Molière...,Grenoble,1673,Molière,mascarille,396
52296,http://catalogue.bnf.fr/ark:/12148/cb309584558,"Le bourgeois gentilhomme, comédie-balet faite ...",Grenoble,1673,Molière,monsieur jourdain,323
4757,http://catalogue.bnf.fr/ark:/12148/cb38651168p,Le bourgeois gentilhomme. Comedie-ballet. Fait...,Grenoble,1671,Molière,m,323


### Only top results

In [8]:
df_person_top_five = df_pers.groupby('target', group_keys=False).apply(lambda x: x.nlargest(5, 'persName_num'))

/var/folders/y2/1xj8qbr56dvf504d6g874cp00000gn/T/ipykernel_68773/3108880041.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_person_top_five = df_pers.groupby('target', group_keys=False).apply(lambda x: x.nlargest(5, 'persName_num'))


In [9]:
df_person_top_five.head(10)

,target,title,pubPlace,date,author,person,persName_num
12961,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Grenoble,1674,"Abeille, Gaspard",argelie,46
12955,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Grenoble,1674,"Abeille, Gaspard",timagène,35
12980,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Grenoble,1674,"Abeille, Gaspard",arcas,16
12984,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Grenoble,1674,"Abeille, Gaspard",phœnix,15
12978,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Grenoble,1674,"Abeille, Gaspard",dione,13
56955,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Grenoble,1676,"Abeille, Gaspard",virgilie,44
56975,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Grenoble,1676,"Abeille, Gaspard",coriolan,32
56948,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Grenoble,1676,"Abeille, Gaspard",camille,30
56986,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Grenoble,1676,"Abeille, Gaspard",aufide,28
56983,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Grenoble,1676,"Abeille, Gaspard",albin,23


In [10]:
df_place_top_two = df_place.groupby('target', group_keys=False).apply(lambda x: x.nlargest(2, 'placeName_num'))

/var/folders/y2/1xj8qbr56dvf504d6g874cp00000gn/T/ipykernel_68773/37667048.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_place_top_two = df_place.groupby('target', group_keys=False).apply(lambda x: x.nlargest(2, 'placeName_num'))


In [11]:
df_place_top_two['uuid'] = df_place_top_two['place'].apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_DNS, x)))

In [12]:
df_place_top_two.head(5)

,target,title,pubPlace,date,author,place,placeName_num,uuid
1164,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Grenoble,1674,"Abeille, Gaspard",clytie,10,97de832a-53f8-5e68-bff8-b6acd68630ff
1162,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Grenoble,1674,"Abeille, Gaspard",mycène,2,4496fd5d-0028-59f4-8e0e-91872008ae6a
5039,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Grenoble,1676,"Abeille, Gaspard",rome,59,cbef6bfb-ce53-50a5-a154-fb3e13aa618e
5038,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Grenoble,1676,"Abeille, Gaspard",lrin,2,0efc0bf9-50b2-5d36-a979-d8d9c1173a55
1656,http://catalogue.bnf.fr/ark:/12148/cb30040061d,"Dipné, infante d'Irlande , tragédie, dédiée à ...",Grenoble,1668,"Aure, François d'",mogale,5,228538c2-3a68-5984-8bf2-3880e2f7747c


#### sample creation (testing purpose)

In [13]:
df_place_top_two_sample = df_place_top_two.head(50)

In [14]:
df_person_top_five_sample = df_person_top_five.head(50)

### Sample: Combine Persons and Places together

In [15]:
columns_to_keep = ['target', 'title', 'pubPlace', 'date', 'author']

In [16]:
# using sample df here, to change
df_pers_place = pd.concat([df_person_top_five_sample, df_place_top_two_sample], ignore_index=True)

In [17]:
df_sample = df_pers_place[columns_to_keep + ['place', 'uuid', 'placeName_num', 'person', 'persName_num']]

## Llama 3.2 

### Take place and persons, but no title of the play

In [18]:
# Function to query Ollama for each target
def get_timeframe_for_play(group):
    target = group['target'].iloc[0]  
    places = ", ".join(group['place'].dropna().astype(str).unique())  # concat unique places for the same target
    persons = ", ".join(group['person'].dropna().astype(str).unique())  # concat unique persons for the same target

    # prompt
    prompt = (
        f"I have a play with the id {target} happening in these places: {places}, "
        f"and with these characters: {persons}. Given this information, suggest me what could be the timeframe where the play takes place. "
        f"I do not need the precise and exact date, but the timespan, formatted in ISO, of the period where the play could have taken place. "
        f"If the information provided are not enough to determine a timeframe, give me your best guess. Respond only with valid JSON."
        f"Do not write an introduction or summary. For the JSON use the form:\n\n"
        f"{{\n  \"reason\": \"why has been chosen\",\n \"period\": \"period_identified\",\n  \"timeframe_start\": \"ISO value of the start\",\n  \"timeframe_end\": \"ISO value of the end\"\n}}\n\n"
        f"The values in the JSON \"timeframe_start\" and \"timeframe_end\" should always be a single valid ISO date in the form YYYY-MM-DD"

    )


    try:
        stream = chat(
            model='llama3.2',
            messages=[{'role': 'user', 'content': prompt}],
            stream=True,
        )

        response_data = ""
        for chunk in stream:
            if 'message' in chunk and 'content' in chunk['message']:
                response_data += chunk['message']['content']

        print(f"Full response for target {target}: {response_data}")

        response_json = json.loads(response_data.strip())

        return {
            "target": target,
            "reason": response_json.get("reason"),
            "period": response_json.get("period"),
            "time_llama_start": response_json.get("timeframe_start"),
            "time_llama_end": response_json.get("timeframe_end"),
        }

    except json.JSONDecodeError as e:
        print(f"JSON decode error for target {target}: {e}")
        print(f"Response data: {response_data}")
        return {
            "target": target,
            "reason": None,
            "period": None,
            "time_llama_start": None,
            "time_llama_end": None,
        }
    except Exception as e:
        print(f"Error for target {target}: {e}")
        return {
            "target": target,
            "reason": None,
            "period": None,
            "time_llama_start": None,
            "time_llama_end": None,
        }



In [21]:
results = []
for target, group in df_sample.groupby('target'):
    result = get_timeframe_for_play(group)
    if result:
        results.append(result)

Full response for target http://catalogue.bnf.fr/ark:/12148/cb30000572m: {
  "reason": "Geographical locations and mythological characters suggest a Late Helladic period, which corresponds to the Mycenaean civilization.",
  "period": "Late Helladic",
  "timeframe_start": "1200-1150 BCE",
  "timeframe_end": "1100-1050 BCE"
}
Full response for target http://catalogue.bnf.fr/ark:/12148/cb30000577b: {
  "reason": "Given that Coriolanus is a play by William Shakespeare, the timeframe can be associated with his life and works.",
  "period": "Roman Republic",
  "timeframe_start": "410-396 BCE",
  "timeframe_end": "395 BCE"
}
Full response for target http://catalogue.bnf.fr/ark:/12148/cb30040061d: {
  "reason": "Geographic locations suggest a Mediterranean or Middle Eastern setting, which often corresponds to ancient periods.",
  "period": "Ancient",
  "timeframe_start": "750-0BCE",
  "timeframe_end": "300-0BCE"
}
Full response for target http://catalogue.bnf.fr/ark:/12148/cb30040062r: {
  "re

In [22]:
results_df_ollama_no_title = pd.DataFrame(results)

In [23]:
results_df_ollama_no_title.head()

,target,reason,period,time_llama_start,time_llama_end
0,http://catalogue.bnf.fr/ark:/12148/cb30000572m,Geographical locations and mythological charac...,Late Helladic,1200-1150 BCE,1100-1050 BCE
1,http://catalogue.bnf.fr/ark:/12148/cb30000577b,Given that Coriolanus is a play by William Sha...,Roman Republic,410-396 BCE,395 BCE
2,http://catalogue.bnf.fr/ark:/12148/cb30040061d,Geographic locations suggest a Mediterranean o...,Ancient,750-0BCE,300-0BCE
3,http://catalogue.bnf.fr/ark:/12148/cb30040062r,Georges Bataille was a French literary figure ...,20th century,1890-01-01,1933-12-31
4,http://catalogue.bnf.fr/ark:/12148/cb30041092x,"Character's age, cultural references, and geog...",Modern,1789-01-01,1850-12-31


### Take title, persons, places

In [24]:
# Function to query Ollama with title
def get_timeframe_for_play_title(group):
    target = group['target'].iloc[0]  
    title = ", ".join(group['title'].dropna().astype(str).unique())  # add title
    places = ", ".join(group['place'].dropna().astype(str).unique())  # concat unique places for the same target
    persons = ", ".join(group['person'].dropna().astype(str).unique())  # concat unique persons for the same target

    # prompt
    prompt = (
        f"I have a play titled {title} with the id {target} happening in these places: {places}, "
        f"and with these characters: {persons}. Given this information, suggest me what could be the timeframe where the play takes place. "
        f"I do not need the precise and exact date, but the timespan, formatted in ISO, of the period where the play could have taken place. "
        f"If the information provided are not enough to determine a timeframe, give me your best guess. Respond only with valid JSON. "
        f"Do not write an introduction or summary. For the JSON use the form:\n\n"
        f"{{\n  \"reason\": \"why has been chosen\",\n \"period\": \"period_identified\",\n  \"timeframe_start\": \"ISO value of the start\",\n  \"timeframe_end\": \"ISO value of the end\"\n}}\n\n"
        f"The values in the JSON \"timeframe_start\" and \"timeframe_end\" should always be a single valid ISO date in the form YYYY-MM-DD."
    )

    try:
        stream = chat(
            model='llama3.2',
            messages=[{'role': 'user', 'content': prompt}],
            stream=True,
        )

        # Collect the streamed response
        response_data = ""
        for chunk in stream:
            if 'message' in chunk and 'content' in chunk['message']:
                response_data += chunk['message']['content']

        # Debug assembled response
        print(f"Full response for target {target}: {response_data}")

        # Attempt to parse the assembled response as JSON
        response_json = json.loads(response_data.strip())

        # Extract the relevant fields
        return {
            "target": target,
            "title": title,
            "reason": response_json.get("reason"),
            "period": response_json.get("period"),
            "time_llama_start": response_json.get("timeframe_start"),
            "time_llama_end": response_json.get("timeframe_end"),
        }

    except json.JSONDecodeError as e:
        print(f"JSON decode error for target {target}: {e}")
        print(f"Response data: {response_data}")
        return {
            "target": target,
            "title": title,
            "reason": None,
            "period": None,
            "time_llama_start": None,
            "time_llama_end": None,
        }
    except Exception as e:
        print(f"Error for target {target}: {e}")
        return {
            "target": target,
            "title": title,
            "reason": None,
            "period": None,
            "time_llama_start": None,
            "time_llama_end": None,
        }

In [26]:
results = []
for target, group in df_sample.groupby('target'):
    result = get_timeframe_for_play_title(group)
    if result:
        results.append(result)

Full response for target http://catalogue.bnf.fr/ark:/12148/cb30000572m: {"reason": "Mythological periods where Clytie was associated with Myc\u00e9ne", "period": "Mycenaean period", "timeframe_start": "1200-1150 BC", "timeframe_end": "1100-1050 BC"}
Full response for target http://catalogue.bnf.fr/ark:/12148/cb30000577b: {
  "reason": "Given that Coriolan is attributed to M. Abeille, a French writer, and the play takes place in Rome, Lutèce (Lrin), it is likely that the timeframe is from the late Middle Ages or Early Modern Period, when the city of Paris was also known as Lutèce.",
  "period": "Late Middle Ages/Early Modern Period",
  "timeframe_start": "1450-01-01",
  "timeframe_end": "1550-12-31"
}
Full response for target http://catalogue.bnf.fr/ark:/12148/cb30040061d: {
  "reason": "Irlande was part of the feudal kingdom of Morgawr (Mogale) during the 5th to 12th centuries, and it shares cultural similarities with 11th-century Normandy",
  "period": "Middle Ages",
  "timeframe_sta

In [27]:
results_df_with_title_llama = pd.DataFrame(results)

In [28]:
results_df_with_title_llama.head()

,target,title,reason,period,time_llama_start,time_llama_end
0,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",Mythological periods where Clytie was associat...,Mycenaean period,1200-1150 BC,1100-1050 BC
1,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",Given that Coriolan is attributed to M. Abeill...,Late Middle Ages/Early Modern Period,1450-01-01,1550-12-31
2,http://catalogue.bnf.fr/ark:/12148/cb30040061d,"Dipné, infante d'Irlande , tragédie, dédiée à ...",Irlande was part of the feudal kingdom of Morg...,Middle Ages,870-1000,1200
3,http://catalogue.bnf.fr/ark:/12148/cb30040062r,"Geneviève, ou L'innocence reconnue , tragédie,...",Geographic locations and historical figures su...,Rococo period,1750-01-01,1780-12-31
4,http://catalogue.bnf.fr/ark:/12148/cb30041092x,La Dorinde du Sr Auvray . Tragi-comédie. Dédié...,Geographical locations and title suggest Baroq...,Baroque,1650-01-01,1700-12-31


In [ ]:
#results_df_with_title_llama.to_csv('/Users/nicola/Downloads/results_with_llama.csv', index=False)

## Open AI API

### Take title, persons, places

In [38]:
load_dotenv()

True

In [39]:
def get_timeframe_for_play(group):
    # add API key
    api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    
    target = group['target'].iloc[0]
    title = ", ".join(group['title'].dropna().astype(str).unique())
    places = ", ".join(group['place'].dropna().astype(str).unique()) # concat unique places for the same target
    persons = ", ".join(group['person'].dropna().astype(str).unique()) # concat unique persons for the same target
    
    prompt = (
        f"I have a play titled {title} with the id {target} happening in these places: {places}, "
        f"and with these characters: {persons}. Given this information, suggest me what could be the timeframe where the play takes place. "
        f"I do not need the precise and exact date, but the timespan, formatted in ISO, of the period where the play could have taken place. "
        f"If the information provided are not enough to determine a timeframe, give me your best guess. Respond only with valid JSON. "
        f"Do not write an introduction or summary. For the JSON use the form:\n\n"
        f"{{\n  \"reason\": \"why has been chosen\",\n \"period\": \"period_identified\",\n  \"timeframe_start\": \"ISO value of the start\",\n  \"timeframe_end\": \"ISO value of the end\"\n}}\n\n"
        f"The values in the JSON \"timeframe_start\" and \"timeframe_end\" should always be a single valid ISO date in the form YYYY-MM-DD."
    )
    
    try:
        completion = client.chat.completions.create(
            model="gpt-4",  # model name, using just gpt-4 here
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        
        response_data = completion.choices[0].message.content.strip()
        response_data = response_data.strip("```").strip("json").strip()
        print(f"Full response for target {target}: {response_data}")
        response_json = json.loads(response_data)
        
        return {
            "target": target,
            "title": title,
            "reason": response_json.get("reason"),
            "period": response_json.get("period"),
            "time_openai_start": response_json.get("timeframe_start"),
            "time_openai_end": response_json.get("timeframe_end"),
        }
        
    except json.JSONDecodeError as e:
        print(f"JSON decode error for target {target}: {e}")
        print(f"Response data: {response_data}")
        return {
            "target": target,
            "title": title,
            "reason": None,
            "period": None,
            "time_openai_start": None,
            "time_openai_end": None,
        }
    except Exception as e:
        print(f"Error for target {target}: {e}")
        return {
            "target": target,
            "title": title,
            "reason": None,
            "period": None,
            "time_openai_start": None,
            "time_openai_end": None,
        }

In [ ]:
results = []
for target, group in df_sample.groupby('target'):
    result = get_timeframe_for_play(group)
    if result:
        results.append(result)

In [35]:
results_df_openai = pd.DataFrame(results)

In [36]:
results_df_openai.head()

,target,title,reason,period,time_openai_start,time_openai_end
0,http://catalogue.bnf.fr/ark:/12148/cb30000572m,"Argélie, reyne de Thessalie . Tragédie",The play is based on Greek mythology and Mycen...,Bronze Age,1600-01-01,1100-12-31
1,http://catalogue.bnf.fr/ark:/12148/cb30000577b,"Coriolan , tragédie. Par M. Abeille",The play Coriolan is a tragedy written by M. A...,Roman Republic,-509-01-01,-27-01-01
2,http://catalogue.bnf.fr/ark:/12148/cb30040061d,"Dipné, infante d'Irlande , tragédie, dédiée à ...","The play Dipné, infante d'Irlande is likely se...",Middle Ages,500-01-01,1500-12-31
3,http://catalogue.bnf.fr/ark:/12148/cb30040062r,"Geneviève, ou L'innocence reconnue , tragédie,...","Based on the information provided, the exact t...",The 17th century,1600-01-01,1699-12-31
4,http://catalogue.bnf.fr/ark:/12148/cb30041092x,La Dorinde du Sr Auvray . Tragi-comédie. Dédié...,As there's no specific date related to the pla...,17th century,1600-01-01,1699-12-31
